In [1]:
import pandas as pd
import datetime

In [3]:
file = pd.read_excel('../Data/KW47_V00.xlsx')

In [116]:
shift_date = file.iloc[4:6, 12: 31: 3].copy()
shift_date.iloc[0, :] = shift_date.iloc[0, :].apply(lambda x: x.strftime("%d %b %Y"))
shift_date = shift_date.apply(lambda x: f"{x.iloc[1]} - {x.iloc[0]}", axis=0)
shift_dates = list(shift_date)

In [99]:
shift_date

,Unnamed: 12,Unnamed: 15,Unnamed: 18,Unnamed: 21,Unnamed: 24,Unnamed: 27,Unnamed: 30
4,21 Nov 2022,22 Nov 2022,23 Nov 2022,24 Nov 2022,25 Nov 2022,26 Nov 2022,27 Nov 2022
5,Pazartesi,Salı,Çarşamba,Perşembe,Cuma,Cumartesi,Pazar


In [96]:
file.iloc[3:5, 8]

3    Güncelleme zamanı
4    18.11.2022  09:20
Name: Unnamed: 8, dtype: object

In [21]:
file.iloc[3:5, 7]

3    Versiyon
4          00
Name: MOE1 Üretim Sıralaması, dtype: object

In [4]:
indices = file.iloc[:, [0, 7, 8, 11]].reset_index()
days = file.iloc[:, 12: 33].reset_index()
combined = pd.concat([indices, days], axis=1).iloc[2:, :]

In [5]:
pipes = pd.read_excel('Cihazlar - Borular.xlsx')

In [6]:
pipes.head()

,Cihaz,Boru,Miktar
0,010130,11053,1
1,010130,11054,1
2,010130,11055,1
3,010130,11056,1
4,010130,11057,1


In [7]:
combined.head()

,index,2022-11-18 09:42:16.870000,MOE1 Üretim Sıralaması,Unnamed: 8,Unnamed: 11,index,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
2,2,NaN,Haftalık toplam,Vardiyalık toplam üretim,>>,2,1144,1214.0,1147.0,1244,...,1059.0,1111,1009.0,1080.0,1221,1320.0,1198.0,1119,0.0,0.0
3,3,NaN,Versiyon,Güncelleme zamanı,NaN,3,1,1.0,1.0,1,...,1.0,1,1.0,1.0,2,2.0,1.0,1,0.0,0.0
4,4,NaN,00,18.11.2022 09:20,NaN,4,2022-11-21 00:00:00,NaN,NaN,2022-11-22 00:00:00,...,NaN,2022-11-25 00:00:00,NaN,NaN,2022-11-26 00:00:00,NaN,NaN,2022-11-27 00:00:00,NaN,NaN
5,5,NaN,NaN,NaN,NaN,5,Pazartesi,NaN,NaN,Salı,...,NaN,Cuma,NaN,NaN,Cumartesi,NaN,NaN,Pazar,NaN,NaN
6,6,NaN,Cihaz TTNr,Cihaz Aile,Adet,6,1,2.0,3.0,1,...,3.0,1,2.0,3.0,1,2.0,3.0,1,2.0,3.0


In [8]:
combined = combined[combined.iloc[:, 1].notna()]

In [9]:
combined = combined[combined.iloc[:, 2].notna()]
combined.iloc[:, 2] = combined.iloc[:, 2].astype("str")
combined = combined[combined.iloc[:, 2].apply(str.isnumeric)]

In [10]:
combined = combined[combined.iloc[:, 6].apply(lambda x: (type(x) != datetime.datetime) and (type(x) != str))]
combined = combined[combined.iloc[:, 3].notna()]

In [11]:
combined.drop('index', axis=1, inplace=True)
combined.reset_index(drop=True, inplace=True)

In [12]:
initial_indices = ["Hat", "Cihaz TTNr", "Cihaz Aile", "Adet"]

In [13]:
week_days = ["Pazartesi", "Salı", "Çarşamba", "Perşembe", "Cuma", "Cumartesi", "Pazar"]
shifts = ["1", "2", "3"]

In [14]:
final_indices = [" ".join([i, j]) for i in week_days for j in shifts]
initial_indices.extend(final_indices)

In [15]:
combined.columns = initial_indices

In [16]:
combined = combined.merge(pipes, left_on="Cihaz TTNr", right_on="Cihaz", how="inner")
combined.drop("Cihaz", axis=1, inplace=True)
combined.insert(3, 'Boru', combined.pop('Boru'))

In [17]:
combined.copy()

,Hat,Cihaz TTNr,Cihaz Aile,Boru,Adet,Pazartesi 1,Pazartesi 2,Pazartesi 3,Salı 1,Salı 2,...,Cuma 1,Cuma 2,Cuma 3,Cumartesi 1,Cumartesi 2,Cumartesi 3,Pazar 1,Pazar 2,Pazar 3,Miktar
0,Yalın 7,600373,CONDENS 4000,683873,1896,258,258.0,245.0,258,258.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Yalın 7,600373,CONDENS 4000,113919,1896,258,258.0,245.0,258,258.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Yalın 7,600373,CONDENS 4000,653366,1896,258,258.0,245.0,258,258.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Yalın 7,600373,CONDENS 4000,651804,1896,258,258.0,245.0,258,258.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Yalın 7,600373,CONDENS 4000,698387,1896,258,258.0,245.0,258,258.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,Yalın 6,902134,CONDENS 7001 C6,900249,8,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,1
1164,Yalın 6,902134,CONDENS 7001 C6,658312,8,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,1
1165,Yalın 6,902134,CONDENS 7001 C6,659217,8,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,1
1166,Yalın 6,902134,CONDENS 7001 C6,658319,8,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,1


In [18]:
for i in range(combined.shape[0]):
    combined.loc[i, "Pazartesi 1": "Pazar 3"] *= combined.loc[i, "Miktar"]

In [19]:
combined.drop("Miktar", axis=1, inplace=True)
combined.drop("Adet", axis=1, inplace=True)

In [20]:
combined.set_index("Hat", inplace=True)

In [21]:
combined

,Cihaz TTNr,Cihaz Aile,Boru,Pazartesi 1,Pazartesi 2,Pazartesi 3,Salı 1,Salı 2,Salı 3,Çarşamba 1,...,Perşembe 3,Cuma 1,Cuma 2,Cuma 3,Cumartesi 1,Cumartesi 2,Cumartesi 3,Pazar 1,Pazar 2,Pazar 3
Hat,,,,,,,,,,,,,,,,,,,,,
Yalın 7,600373,CONDENS 4000,683873,258,258.0,245.0,258,258.0,245.0,258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yalın 7,600373,CONDENS 4000,113919,258,258.0,245.0,258,258.0,245.0,258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yalın 7,600373,CONDENS 4000,653366,258,258.0,245.0,258,258.0,245.0,258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yalın 7,600373,CONDENS 4000,651804,258,258.0,245.0,258,258.0,245.0,258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yalın 7,600373,CONDENS 4000,698387,258,258.0,245.0,258,258.0,245.0,258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yalın 6,902134,CONDENS 7001 C6,900249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN
Yalın 6,902134,CONDENS 7001 C6,658312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN
Yalın 6,902134,CONDENS 7001 C6,659217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
combined.to_excel("initial.xlsx")